In [ ]:
import pandas as pd

In [ ]:
augmented_inbetween = pd.read_csv('data/interim/20241105_data_augmentation/augmented_data.csv',index_col=0)
augmented_inbetween_meta = pd.read_csv('data/interim/20241105_data_augmentation/augmented_clinical.csv',index_col=0)
print(augmented_inbetween.shape, augmented_inbetween_meta.shape)

In [ ]:
augmented_inbetween_meta.value_counts()

In [ ]:
augmented_g3 = pd.read_csv('data/interim/20241105_data_augmentation/group3/augmented_data.csv',index_col=0)
augmented_g3_meta = pd.read_csv('data/interim/20241105_data_augmentation/group3/augmented_clinical.csv',index_col=0)
print(augmented_g3.shape, augmented_g3_meta.shape)

In [ ]:
augmented_g3_meta.value_counts()

In [ ]:
augmented_g4 = pd.read_csv('data/interim/20241105_data_augmentation/group4/augmented_data.csv',index_col=0)
augmented_g4_meta = pd.read_csv('data/interim/20241105_data_augmentation/group4/augmented_clinical.csv',index_col=0)
print(augmented_g4.shape, augmented_g4_meta.shape)

In [ ]:
augmented_g4_meta.value_counts()

In [ ]:
cavalli_from_geoquery = pd.read_csv('data/raw/cavalli_from_geoquery.csv',index_col=0)
cavalli_from_geoquery_meta = pd.read_csv('data/raw/cavalli_from_geoquery_metadata.csv',index_col=0)
print(cavalli_from_geoquery.shape, cavalli_from_geoquery_meta.shape)

# Consensus Clustering Results

In [ ]:
import os
import pandas as pd
from src.visualization.visualize import plot_umap

In [ ]:
def load_data(data_path,metadata_path,consensus_path):
    data = pd.read_csv(data_path,index_col=0)
    metadata = pd.read_csv(metadata_path,index_col=0).replace({'Group3':'Group 3','Group4':'Group 4'})
    metadata=pd.Series(metadata.values.flatten(), index=metadata.index, name='group')
    consensus = pd.DataFrame()
    for k_here in range(2, 4):
        consensus_i = pd.read_csv(os.path.join(consensus_path, f'consensusClass_k{k_here}.csv'))
        consensus[f'k_{k_here}'] = consensus_i['x']
    return data, metadata, consensus

In [ ]:
def get_consensus(consensus, metadata):
    consensus_k2 = metadata.copy()
    consensus_k2.loc[consensus.index] = consensus['k_2']
    consensus_k3 = metadata.copy()
    consensus_k3.loc[consensus.index] = consensus['k_3']
    return consensus_k2, consensus_k3

In [ ]:
def find_overlap(metadata, consensus_k2, consensus_k3):
    # According to Cavalli, to find the overlap, they
    # "counted the number of samples that were initially considered to be of a particular
    # subgroup for k=2 and moved to be in another subgroup at k=3"
    # For k=2, 1: Group 3, 2: Group 4
    k2_dict = {1: 'Group 3', 2: 'Group 4'}
    consensus_k2.replace(k2_dict, inplace=True)
    # For k=3, 2 and 3: Group 4, 1: Group 3
    k3_dict = {2: 'Group 4', 3: 'Group 4', 1: 'Group 3'}
    consensus_k3.replace(k3_dict, inplace=True)
    # Contingency table between k=2 and k=3
    cross_tab_k2_k3=pd.crosstab(consensus_k2, consensus_k3, margins=True)
    cross_tab_k2_k3.index.name = 'k=2'
    cross_tab_k2_k3.columns.name = 'k=3'
    # Get the patients that have changed group from consensus_k2_clinical to consensus_k3
    changed_patients_k3_to_k2 = consensus_k2[consensus_k2 != consensus_k3].index
    metadata_changed_k3_to_k2 = metadata.copy()
    metadata_changed_k3_to_k2[changed_patients_k3_to_k2] = 'G3-G4'
    # Get corresponding contingency table
    original_consensus_comparison = pd.crosstab(metadata, metadata_changed_k3_to_k2, margins=True)
    original_consensus_comparison.index.name = 'Original'
    original_consensus_comparison.columns.name = 'ConsensusClustering'
    return cross_tab_k2_k3, metadata_changed_k3_to_k2, original_consensus_comparison

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='Analyze results from ConsensusCluster')
parser.add_argument('--data_path', type=str, help='Path to the data file')
parser.add_argument('--metadata_path', type=str, help='Path to the metadata file')
parser.add_argument('--consensus_path', type=str, help='Path to the directory containing the ConsensusCluster results')
parser.add_argument('--save_path', type=str, help='Path to the directory to save the results')
parser.add_argument('--use_latent', action='store_true', help='Use the latent space instead of the original data')

# Simulate args with sys.argv
import sys
sys.argv = [
    'notebook',
    '--data_path', 'data/raw/cavalli_from_geoquery.csv',
    '--metadata_path', 'data/raw/cavalli_from_geoquery_metadata.csv',
    '--consensus_path', 'data/processed/20241107_consensusclustering/results_real_space/km/',
    '--save_path', 'data/interim/20241105_data_augmentation/consensus',
    # '--use_latent'
    ]
args = parser.parse_args()
print(args)

In [ ]:
# Load data
data, metadata, consensus = load_data(data_path=args.data_path,consensus_path=args.consensus_path,metadata_path=args.metadata_path)
# Get group assignments for k=2 and k=3 from the consensus clustering results
consensus_k2, consensus_k3 = get_consensus(consensus=consensus, metadata=metadata)
# Plot umaps for k=2 and k=3
dict_umap_consensus_k2 = {'SHH': '#b22222', 'WNT': '#6495ed', 1: '#ffd700', 2: '#008000'}
dict_umap_consensus_k3 = {'SHH': '#b22222', 'WNT': '#6495ed', 1: '#ffd700', 2: '#008000', 3: '#ff69b4'}
plot_umap(data, consensus_k2, dict_umap_consensus_k2, n_components=2,save_fig=False,
          save_as=os.path.join(args.save_path, 'k2_latent' if args.use_latent else 'k2_noprepro'),
          seed=2023, title=None,show=False)
plot_umap(data, consensus_k3, dict_umap_consensus_k3, n_components=2, save_fig=False,
          save_as=os.path.join(args.save_path, 'k3_latent' if args.use_latent else 'k3_noprepro'),
          seed=2023, title=None,show=False)

In [ ]:
consensus_k2.value_counts()

In [ ]:
consensus_k3.value_counts()

In [ ]:
# Find overlap between k=2 and k=3
# if args.use_latent:
cross_tab_k2_k3, metadata_changed_k3_to_k2, original_consensus_comparison = find_overlap(
    metadata=metadata, consensus_k2=consensus_k2, consensus_k3=consensus_k3)
#     cross_tab_k2_k3.to_csv(os.path.join(args.save_path, 'contingency_k2_k3.csv'))
#     cross_tab_k2_k3.to_latex(os.path.join(args.save_path, 'contingency_k2_k3.tex'))
#     original_consensus_comparison.to_csv(os.path.join(args.save_path, 'original_consensus_comparison.csv'))
#     original_consensus_comparison.to_latex(os.path.join(args.save_path, 'original_consensus_comparison.tex'))
#     metadata_changed_k3_to_k2.to_csv(os.path.join(args.save_path, 'metadata_changed_k3_to_k2.csv'))
#     # Plot umap with In between groups:
dict_umap = {'SHH': '#b22222', 'WNT': '#6495ed', 'Group 3': '#ffd700', 'Group 4': '#008000', 'G3-G4': '#db7093'}
plot_umap(data, metadata_changed_k3_to_k2, dict_umap, n_components=2, save_fig=False,
          save_as=os.path.join(args.save_path, 'k3_to_k2_latent' if args.use_latent else 'k3_to_k2_noprepro'),
          seed=2023, title=None,show=False)

In [ ]:
metadata.value_counts()

In [ ]:
cross_tab_k2_k3

In [ ]:
metadata_changed_k3_to_k2.value_counts()

In [ ]:
original_consensus_comparison

# kNN bootstrap results

In [ ]:
import pandas as pd
from src.visualization.visualize import plot_umap

In [ ]:
# Load data
data = pd.read_csv('data/interim/20241115_preprocessing/cavalli_maha.csv', index_col=0)
metadata = pd.read_csv('data/processed/20241115_knn_bootstrap/metadata_after_bootstrap.csv', index_col=0).squeeze()
data.shape, metadata.shape

In [ ]:
# save_path = 'data/processed/20241115_knn_bootstrap/knn_bootstrapping/preprocessed_umap/'
# os.makedirs(save_path, exist_ok=True)

In [ ]:
metadata.value_counts()

In [ ]:
dict_umap = {'SHH': '#b22222',
                 'WNT': '#6495ed',
                 'Group 3': '#ffd700',
                 'Group 4': '#008000',
                 'G3-G4': '#db7093'}

In [ ]:
plot_umap(data=data.loc[metadata.index],
          clinical=metadata,
          colors_dict=dict_umap,
          save_fig=False,
          save_as=None,
          seed=2023,
          show=True
          )

# Data augmentation UMAP

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '112'

In [ ]:
import pandas as pd
from src.visualization.visualize import plot_umap

In [ ]:
augmented_data = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_data.csv',index_col=0)
augmented_clinical = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_clinical.csv',index_col=0).squeeze()
print(augmented_data.shape, augmented_clinical.shape)

In [ ]:
augmented_clinical.replace({'synthetic_Group 3':'Synthetic Group 3','synthetic_Group 4':'Synthetic Group 4','synthetic_G3-G4':'Synthetic G3-G4'},inplace=True)

In [ ]:
augmented_clinical.value_counts()

In [ ]:
dict_umap = {
    # 'SHH': '#b22222', 
    # 'WNT': '#6495ed', 
    'Group 3': '#ffd700', 
    'Group 4': '#008000', 
    'G3-G4': '#db7093',
    'Synthetic Group 3': '#917b00',
    'Synthetic Group 4': '#014401',
    'Synthetic G3-G4': '#84445a'
}

In [ ]:
# Remove SHH and WNT patients:
clinical_umap = augmented_clinical[~augmented_clinical.isin(['SHH','WNT'])]
data_umap = augmented_data.loc[clinical_umap.index]
print(data_umap.shape, clinical_umap.shape)

In [ ]:
os.makedirs('reports/figures/20241115_umap_augmented',exist_ok=True)
plot_umap(
    data=data_umap,
    clinical=clinical_umap,
    colors_dict=dict_umap,
    n_components=2,
    save_fig=True,
    save_as='reports/figures/20241115_umap_augmented/umap',
    seed=2023,
    title=None,
    show=True,
)

# Classification

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '112'

In [ ]:
from src.data_processing.classification import classification_benchmark
import numpy as np
from tqdm import tqdm
from matplotlib import colormaps
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd


In [ ]:
save_dir = 'reports/figures/20241115_classification_notebook'
os.makedirs(save_dir,exist_ok=True)

In [ ]:
augmented_data = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_data.csv',index_col=0)
augmented_clinical = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_clinical.csv',index_col=0).squeeze()
print(augmented_data.shape, augmented_clinical.shape)

## Classification of real patients

In [ ]:
save_dir_i = os.path.join(save_dir,'real_g3g4')
os.makedirs(save_dir_i,exist_ok=True)

In [ ]:
# import original data, after preprocessing
data = pd.read_csv('data/interim/20241115_preprocessing/cavalli_maha.csv', index_col=0)
metadata = pd.read_csv('data/raw/GEO/cavalli_subgroups.csv', index_col=0).squeeze()
print(data.shape, metadata.shape)
print(metadata.value_counts())

In [ ]:
# Select data for classification
num_classes = 2
clinical_classification = metadata[metadata.isin(['Group3','Group4'])]
data_classification = data.loc[clinical_classification.index]
print(data_classification.shape, clinical_classification.shape)
print(clinical_classification.value_counts())

In [ ]:
# Classification on weighted data with different seeds
seeds = np.random.randint(0, 1e9, 10)
metrics_list = []
for seed_i in tqdm(seeds):
    weighted_classification = classification_benchmark(
        X_data=data_classification,
        y_data=clinical_classification,
        classification_type='weighted',
        num_classes=num_classes,
        seed=seed_i,
        test_size=0.2,
        n_br=100,
        num_threads=112,
        n_trials=100,
        )
    (model_weights, metrics_weights, y_test_le, y_pred_weights, data_weights, weighted_params) = weighted_classification
    metrics_list.append(metrics_weights)
    metrics_weights.to_csv(os.path.join(save_dir_i,f'metrics_weights_seed_{seed_i}.csv'))

In [ ]:
# Plot mean of metrics
metrics_list = np.array(metrics_list)
mean_metrics = np.median(metrics_list,axis=0)
std_metrics = np.std(metrics_list,axis=0)
legend_labels = metrics_weights.columns
cmap = 'Pastel2'
colors = colormaps[cmap](np.linspace(0, 1, 7))
plt.figure(figsize=(9, 7))
bp=plt.boxplot(metrics_list[:,0], labels=metrics_weights.columns, patch_artist=True)
# Rotate the x-axis labels
plt.xticks(rotation=45,fontsize=14)
plt.yticks(fontsize=14)
plt.ylim([0.,1.05])
plt.xlabel('Score',fontsize=16)
plt.title('Real Group 3 and Group 4',fontsize=18)
# Add legend
# Create a legend
# legend_labels = ['Accuracy', 'Balanced accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
# Set the colors
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
for median in bp['medians']:
    median.set(color='navy', linewidth=2)

# legend_handles = [mpatches.Patch(color=colors[i], label=legend_labels[i]) for i in range(len(legend_labels))]
# plt.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig(os.path.join(save_dir_i,'groups3_and_4_real_recspace_mean_metrics_boxplot.png'),bbox_inches='tight',dpi=600,format='png')
plt.savefig(os.path.join(save_dir_i,'groups3_and_4_real_recspace_mean_metrics_boxplot.svg'),bbox_inches='tight',format='svg')
plt.savefig(os.path.join(save_dir_i,'groups3_and_4_real_recspace_mean_metrics_boxplot.pdf'),bbox_inches='tight',format='pdf')
plt.show()

## Classification of real patients, including G3-G4 group

In [ ]:
save_dir_i = os.path.join(save_dir,'real_g3g4g3-g4')
os.makedirs(save_dir_i,exist_ok=True)

In [ ]:
# Select data for classification
num_classes = 3
clinical_classification = augmented_clinical[augmented_clinical.isin(['Group 3','Group 4','G3-G4'])]
data_classification = data.loc[clinical_classification.index]
print(clinical_classification.shape, data_classification.shape)
print(clinical_classification.value_counts())

In [ ]:
# Classification on weighted data with different seeds
seeds = np.random.randint(0, 1e9, 10)
metrics_list = []
for seed_i in tqdm(seeds):
    weighted_classification = classification_benchmark(
        X_data=data_classification,
        y_data=clinical_classification,
        classification_type='weighted',
        num_classes=num_classes,
        seed=seed_i,
        test_size=0.2,
        n_br=100,
        num_threads=4,
        n_trials=100,
        )
    (model_weights, metrics_weights, y_test_le, y_pred_weights, data_weights, weighted_params) = weighted_classification
    metrics_list.append(metrics_weights)
    metrics_weights.to_csv(os.path.join(save_dir_i,f'metrics_weights_seed_{seed_i}.csv'))

In [ ]:
# Plot mean of metrics
metrics_list = np.array(metrics_list)
mean_metrics = np.mean(metrics_list,axis=0)
std_metrics = np.std(metrics_list,axis=0)
legend_labels = metrics_weights.columns
cmap = 'Pastel2'
colors = colormaps[cmap](np.linspace(0, 1, 7))
plt.figure(figsize=(9, 7))
bp=plt.boxplot(metrics_list[:,0], labels=metrics_weights.columns, patch_artist=True)
# Rotate the x-axis labels
plt.xticks(rotation=45,fontsize=14)
plt.yticks(fontsize=14)
plt.ylim([0.,1.05])
plt.xlabel('Score',fontsize=16)
plt.title('Real Group 3, Group 4, and G3-G4',fontsize=18)
# Add legend
# Create a legend
# legend_labels = ['Accuracy', 'Balanced accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
# Set the colors
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
for median in bp['medians']:
    median.set(color='navy', linewidth=2)
# legend_handles = [mpatches.Patch(color=colors[i], label=legend_labels[i]) for i in range(len(legend_labels))]
# plt.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_inbetween_real_recspace_mean_metrics_boxplot.png'),bbox_inches='tight',dpi=600,format='png')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_inbetween_real_recspace_mean_metrics_boxplot.svg'),bbox_inches='tight',format='svg')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_inbetween_real_recspace_mean_metrics_boxplot.pdf'),bbox_inches='tight',format='pdf')
plt.show()

## Classification of synthetic G3 and G4

In [ ]:
save_dir_i = os.path.join(save_dir,'synth_g3g4')
os.makedirs(save_dir_i,exist_ok=True)

In [ ]:
augmented_clinical.value_counts()

In [ ]:
# Select data for classification
num_classes = 2
clinical_classification = augmented_clinical[augmented_clinical.isin(['synthetic_Group 3','synthetic_Group 4'])]
data_classification = augmented_data.loc[clinical_classification.index]
print(clinical_classification.shape, data_classification.shape)
print(clinical_classification.value_counts())

In [ ]:
# Classification on weighted data with different seeds
seeds = np.random.randint(0, 1e9, 10)
metrics_list = []
for seed_i in tqdm(seeds):
    weighted_classification = classification_benchmark(
        X_data=data_classification,
        y_data=clinical_classification,
        classification_type='weighted',
        num_classes=num_classes,
        seed=seed_i,
        test_size=0.2,
        n_br=100,
        num_threads=112,
        n_trials=100,
        )
    (model_weights, metrics_weights, y_test_le, y_pred_weights, data_weights, weighted_params) = weighted_classification
    metrics_list.append(metrics_weights)
    metrics_weights.to_csv(os.path.join(save_dir_i,f'metrics_weights_seed_{seed_i}.csv'))

In [ ]:
# Plot mean of metrics
metrics_list = np.array(metrics_list)
mean_metrics = np.mean(metrics_list,axis=0)
std_metrics = np.std(metrics_list,axis=0)
legend_labels = metrics_weights.columns
cmap = 'Pastel2'
colors = colormaps[cmap](np.linspace(0, 1, 7))
plt.figure(figsize=(9, 7))
bp=plt.boxplot(metrics_list[:,0], labels=metrics_weights.columns, patch_artist=True)
# Rotate the x-axis labels
plt.xticks(rotation=45,fontsize=14)
plt.yticks(fontsize=14)
plt.ylim([0.,1.05])
plt.xlabel('Score',fontsize=16)
plt.title('Synthetic Group 3 and Group 4',fontsize=18)
# Add legend
# Create a legend
# legend_labels = ['Accuracy', 'Balanced accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
# Set the colors
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
for median in bp['medians']:
    median.set(color='navy', linewidth=2)
# legend_handles = [mpatches.Patch(color=colors[i], label=legend_labels[i]) for i in range(len(legend_labels))]
# plt.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_synth_recspace_mean_metrics_boxplot.png'),bbox_inches='tight',dpi=600,format='png')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_synth_recspace_mean_metrics_boxplot.svg'),bbox_inches='tight',format='svg')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_synth_recspace_mean_metrics_boxplot.pdf'),bbox_inches='tight',format='pdf')
plt.show()

## Classification of synthetic G3, G4, and G3-G4

In [ ]:
save_dir_i = os.path.join(save_dir,'synth_g3g4g3-g4')
os.makedirs(save_dir_i,exist_ok=True)

In [ ]:
augmented_clinical.value_counts()

In [ ]:
# Select data for classification
num_classes = 3
clinical_classification = augmented_clinical[augmented_clinical.isin(['synthetic_Group 3','synthetic_Group 4','synthetic_G3-G4'])]
data_classification = augmented_data.loc[clinical_classification.index]
print(clinical_classification.shape, data_classification.shape)
print(clinical_classification.value_counts())

In [ ]:
# Classification on weighted data with different seeds
seeds = np.random.randint(0, 1e9, 10)
metrics_list = []
for seed_i in tqdm(seeds):
    weighted_classification = classification_benchmark(
        X_data=data_classification,
        y_data=clinical_classification,
        classification_type='weighted',
        num_classes=num_classes,
        seed=seed_i,
        test_size=0.2,
        n_br=100,
        num_threads=112,
        n_trials=100,
        )
    (model_weights, metrics_weights, y_test_le, y_pred_weights, data_weights, weighted_params) = weighted_classification
    metrics_list.append(metrics_weights)
    metrics_weights.to_csv(os.path.join(save_dir_i,f'metrics_weights_seed_{seed_i}.csv'))

In [ ]:
# Plot mean of metrics
metrics_list = np.array(metrics_list)
mean_metrics = np.mean(metrics_list,axis=0)
std_metrics = np.std(metrics_list,axis=0)
legend_labels = metrics_weights.columns
cmap = 'Pastel2'
colors = colormaps[cmap](np.linspace(0, 1, 7))
plt.figure(figsize=(9, 7))
bp=plt.boxplot(metrics_list[:,0], labels=metrics_weights.columns, patch_artist=True)
# Rotate the x-axis labels
plt.xticks(rotation=45,fontsize=14)
plt.yticks(fontsize=14)
plt.ylim([0.,1.05])
plt.xlabel('Score',fontsize=16)
plt.title('Synthetic Group 3, Group 4, and G3-G4', fontsize=18)
# Add legend
# Create a legend
# legend_labels = ['Accuracy', 'Balanced accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
# Set the colors
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
for median in bp['medians']:
    median.set(color='navy', linewidth=2)
# legend_handles = [mpatches.Patch(color=colors[i], label=legend_labels[i]) for i in range(len(legend_labels))]
# plt.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_G3G4_synth_recspace_mean_metrics_boxplot.png'),bbox_inches='tight',dpi=600,format='png')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_G3G4_synth_recspace_mean_metrics_boxplot.svg'),bbox_inches='tight',format='svg')
plt.savefig(os.path.join(save_dir_i,'groups_3_4_G3G4_synth_recspace_mean_metrics_boxplot.pdf'),bbox_inches='tight',format='pdf')
plt.show()

# Getting Medians of classification metrics

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '112'
import pandas as pd

In [ ]:
def list_metrics_seed_files(folder_path):
    # List all files in the given folder
    all_files = os.listdir(folder_path)
    # Filter files that start with 'metrics_seed'
    metrics_seed_files = [f for f in all_files if f.startswith('metrics_seed') or f.startswith('metrics_weights_seed')]
    return metrics_seed_files

In [ ]:
# Example usage
folder_path = 'reports/figures/20241115_classification_original/'
metrics_seed_files = list_metrics_seed_files(folder_path)
print(len(metrics_seed_files))

In [ ]:
# Initialize a list to store the metrics
def all_metrics(folder_path):
    metrics_list = []
    # Loop through each file and append the first row to the list
    for i, file in enumerate(metrics_seed_files):
        metrics_seed_i = pd.read_csv(os.path.join(folder_path, file), index_col=0).iloc[0]  # All rows are the same, so get the first row
        metrics_seed_i.name = i  # Set the index to the current iteration
        metrics_list.append(metrics_seed_i)
    
    # Create a DataFrame from the list
    all_metrics = pd.DataFrame(metrics_list)
    
    # Reset the index to have consecutive integers starting from 0
    all_metrics.reset_index(drop=True, inplace=True)
    
    return all_metrics

In [ ]:
# List of folders to analyze
folders = [
    '20241115_classification_original',
    '20241115_classification_latent',
    '20241115_classification_postprocessed',
    '20241115_classification_notebook/real_g3g4',
    '20241115_classification_notebook/synth_g3g4',
    '20241115_classification_notebook/real_g3g4g3-g4',
    '20241115_classification_notebook/synth_g3g4g3-g4',
]

In [ ]:
medians_list = []
for folder_i in folders:
    folder_path = f'reports/figures/{folder_i}'
    metrics_seed_files = list_metrics_seed_files(folder_path)
    all_metrics_i = all_metrics(folder_path)
    median_i=all_metrics_i.median(axis=0)
    median_i.name=folder_i
    medians_list.append(median_i)
    print(folder_i, all_metrics_i.shape, median_i.shape)
all_medians = pd.DataFrame(medians_list)

In [ ]:
all_medians.index=[
    'Original Data',
    'Latent Space',
    'Reconstructed Data',
    'Real G3 and G4',
    'Synthetic G3 and G4',
    'Real G3, G4, and G3-G4',
    'Synthetic G3, G4, and G3-G4',
]
all_medians.to_csv('reports/figures/20241115_classification_medians.csv')
all_medians.to_latex('reports/figures/20241115_classification_medians.tex')

In [ ]:
all_medians

In [ ]:
print(all_medians.loc['Synthetic G3 and G4'])

In [ ]:
print(all_medians.loc['Synthetic G3, G4, and G3-G4'])

In [ ]:
print(all_medians.loc['Synthetic G3 and G4']-all_medians.loc['Synthetic G3, G4, and G3-G4'])
print('Average performance loss:',(all_medians.loc['Synthetic G3 and G4']-all_medians.loc['Synthetic G3, G4, and G3-G4']).mean())

# Getting patients missclasified

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '112'

In [ ]:
from src.data_processing.classification import classification_benchmark
import numpy as np
from tqdm import tqdm
from matplotlib import colormaps
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd

In [ ]:
original_data = pd.read_csv('data/raw/GEOquery/GSE85217_expression_data.csv',index_col=0).T
original_metadata = pd.read_csv('data/raw/GEO/cavalli_subgroups.csv',index_col=0).squeeze()
print(original_metadata.shape, original_data.shape)
print(original_metadata.value_counts())

In [ ]:
# Select data for classification
num_classes = 4
clinical_classification = original_metadata
data_classification = original_data.loc[clinical_classification.index]
print(clinical_classification.shape, data_classification.shape)
print(clinical_classification.value_counts())

In [ ]:
# Classification on weighted data with different seeds
seeds = np.random.randint(0, 1e9, 10)
metrics_list = []
misclassified_patients = []
for seed_i in tqdm(seeds):
    weighted_classification = classification_benchmark(
        X_data=data_classification,
        y_data=clinical_classification,
        classification_type='weighted',
        num_classes=num_classes,
        seed=seed_i,
        test_size=0.2,
        n_br=100,
        num_threads=112,
        n_trials=100,
        )
    (model_weights, metrics_weights, y_test_le, y_pred_weights, data_weights, weighted_params) = weighted_classification
    metrics_list.append(metrics_weights)
    # Get patients wrongly predicted:
    y_pred_labels_i = np.argmax(y_pred_weights, axis=1) # Convert y_pred_weights to predicted class labels
    misclassified_indices_i = np.where(y_test_le != y_pred_labels_i)[0] # Find the indices of the misclassified patients
    misclassified_patients_i = clinical_classification.index[misclassified_indices_i] # Get the misclassified patients
    print("Misclassified patients:", misclassified_patients_i.shape[0])
    misclassified_patients.append(misclassified_patients_i)
    # metrics_weights.to_csv(os.path.join(save_dir_i,f'metrics_weights_seed_{seed_i}.csv'))

In [ ]:
# Plot mean of metrics
metrics_list = np.array(metrics_list)
mean_metrics = np.mean(metrics_list,axis=0)
std_metrics = np.std(metrics_list,axis=0)
legend_labels = metrics_weights.columns
cmap = 'Pastel2'
colors = colormaps[cmap](np.linspace(0, 1, 7))
plt.figure(figsize=(9, 7))
bp=plt.boxplot(metrics_list[:,0], labels=metrics_weights.columns, patch_artist=True)
# Rotate the x-axis labels
plt.xticks(rotation=45,fontsize=14)
plt.yticks(fontsize=14)
plt.ylim([0.,1.05])
plt.xlabel('Score',fontsize=16)
plt.title('Synthetic Group 3, Group 4, and G3-G4', fontsize=18)
# Add legend
# Create a legend
# legend_labels = ['Accuracy', 'Balanced accuracy', 'F1 score', 'Precision', 'Recall', 'ROC AUC', 'PR AUC']
# Set the colors
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
for median in bp['medians']:
    median.set(color='navy', linewidth=2)
# legend_handles = [mpatches.Patch(color=colors[i], label=legend_labels[i]) for i in range(len(legend_labels))]
# plt.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.savefig(os.path.join(save_dir_i,'groups_3_4_G3G4_synth_recspace_mean_metrics_boxplot.png'),bbox_inches='tight',dpi=600,format='png')
# plt.savefig(os.path.join(save_dir_i,'groups_3_4_G3G4_synth_recspace_mean_metrics_boxplot.svg'),bbox_inches='tight',format='svg')
# plt.savefig(os.path.join(save_dir_i,'groups_3_4_G3G4_synth_recspace_mean_metrics_boxplot.pdf'),bbox_inches='tight',format='pdf')
plt.show()

In [ ]:
all_missed_patients=[item for sublist in misclassified_patients for item in sublist]
unique_missed_patients,count_missed_patients=np.unique(all_missed_patients,return_counts=True)
df_missed=pd.Series(count_missed_patients,index=unique_missed_patients,name='Missclassified_Patients')
df_missed.value_counts()

In [ ]:
df_missed=clinical_classification[unique_missed_patients]
df_missed.value_counts()

In [ ]:
clinical_missed = clinical_classification.copy()
clinical_missed.loc[df_missed.index] = 'Misclassified'
clinical_missed.value_counts()

In [ ]:
# Load kNN bootstrap results
metadata_knn_bootstrap = pd.read_csv('data/processed/20241115_knn_bootstrap/metadata_after_bootstrap.csv',index_col=0)
metadata_knn_bootstrap = metadata_knn_bootstrap.squeeze()
metadata_knn_bootstrap.value_counts()

In [ ]:
# Load consensus clustering results
metadata_consensus_clustering = pd.read_csv('data/processed/20241115_consensusclustering/results_real_space/km/metadata_changed_k3_to_k2.csv',index_col=0)
metadata_consensus_clustering = metadata_consensus_clustering.squeeze()
metadata_consensus_clustering.value_counts()

In [ ]:
# Contingency table between missclassified and kNN bootstrap
cross_tab_missed_knn = pd.crosstab(clinical_missed, metadata_knn_bootstrap, margins=True,)
cross_tab_missed_knn.index.name = None
cross_tab_missed_knn

In [ ]:
# Contingency table between missclassified and kNN consensus clustering
cross_tab_missed_consensus = pd.crosstab(clinical_missed, metadata_consensus_clustering, margins=True,)
cross_tab_missed_consensus.index.name = None
cross_tab_missed_consensus

In [ ]:
save_path_missed='reports/figures/20241115_classification_original/misclassified'
os.makedirs(save_path_missed,exist_ok=True)
df_missed.to_csv(os.path.join(save_path_missed,'df_missed.csv'))
clinical_missed.to_csv(os.path.join(save_path_missed,'clinical_missed.csv'))
cross_tab_missed_knn.to_csv(os.path.join(save_path_missed,'cross_tab_missed_knn.csv'))
cross_tab_missed_consensus.to_csv(os.path.join(save_path_missed,'cross_tab_missed_consensus.csv'))

# Comparing Consensus Clustering and kNN bootstrap

In [ ]:
import pandas as pd

In [ ]:
# Load consensus clustering results
metadata_consensus_clustering = pd.read_csv('data/processed/20241115_consensusclustering/results_real_space/km/metadata_changed_k3_to_k2.csv',index_col=0)
metadata_consensus_clustering = metadata_consensus_clustering.squeeze()
metadata_consensus_clustering.value_counts()

In [ ]:
# Load consensus clustering results with preprocessed data
metadata_consensus_clustering_preprocessed = pd.read_csv('data/processed/20241115_consensusclustering/results_preprocessed/km/metadata_changed_k3_to_k2.csv',index_col=0)
metadata_consensus_clustering_preprocessed = metadata_consensus_clustering_preprocessed.squeeze()
metadata_consensus_clustering_preprocessed.value_counts()

In [ ]:
# Load kNN bootstrap results
metadata_knn_bootstrap = pd.read_csv('data/processed/20241115_knn_bootstrap_preprocessed/metadata_after_bootstrap.csv',index_col=0)
metadata_knn_bootstrap = metadata_knn_bootstrap.squeeze()
metadata_knn_bootstrap.value_counts()

In [ ]:
# Contingency table between consensus clustering and kNN bootstrap
cross_tab_consensus_knn = pd.crosstab(metadata_consensus_clustering, metadata_knn_bootstrap, margins=True)
cross_tab_consensus_knn.index.name = None
cross_tab_consensus_knn

In [ ]:
# Preview export to latex:
latex_table = cross_tab_consensus_knn.to_latex(escape=False,index=True)
latex_table = latex_table.replace('Sample_characteristics_ch1', '\\diagbox{kNN}{CC}')
print(latex_table)

In [ ]:
# Contingency table between consensus clustering with preprocessed data and kNN bootstrap
cross_tab_consensus_knn_preprocessed = pd.crosstab(metadata_consensus_clustering_preprocessed, metadata_knn_bootstrap, margins=True)
cross_tab_consensus_knn_preprocessed.index.name = None
cross_tab_consensus_knn_preprocessed

In [ ]:
# Preview export to latex:
latex_table = cross_tab_consensus_knn_preprocessed.to_latex(escape=False,index=True)
latex_table = latex_table.replace('Sample_characteristics_ch1', '\\diagbox{kNN}{CC prepro}')
print(latex_table)

# Genes differentially expressed comparison with Northcott et al. 2019

In [ ]:
import os
import pandas as pd
from src.kruskalwallis_inbetween import plot_differential_genes

In [ ]:
path_save_figs = 'reports/figures/20241115_kw/boxplot_augmented/synth_patients/northcott2019_genes'
os.makedirs(path_save_figs, exist_ok=True)

In [ ]:
# Load gene correspondence from original data:
gene_correspondence = pd.read_csv('data/raw/GEO/cavalli_gene_correspondence.csv', index_col=0)

In [ ]:
# Load augmented data
data = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_data.csv', index_col=0) 
metadata = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_clinical.csv', index_col=0).squeeze()
data.shape, metadata.shape

In [ ]:
# Select only synthetic patients
data = data.loc[metadata[metadata.str.contains('synthetic')].index]
metadata = metadata[metadata.str.contains('synthetic')].replace({'synthetic_Group 3':'Group3','synthetic_Group 4':'Group4','synthetic_G3-G4':'G3-G4'})
data.shape, metadata.shape

In [ ]:
# Load p_values
p_values = pd.read_csv('data/processed/20241115_differentially_expressed_genes/synth_patients/p_values_dunn.csv', index_col=0)

In [ ]:
# Change column names from data, and indices from p_values, which are in ENSG format, to gene symbols:
data.columns=gene_correspondence.loc[data.columns]['HGNC_symbol_from_ensemblv77'].to_list()
p_values.index=gene_correspondence.loc[p_values.index]['HGNC_symbol_from_ensemblv77'].to_list()

In [ ]:
# Get genes differentially expressed that coincide with Northcott et al. 2019
northcott_genes = pd.read_csv('data/processed/20241115_genes_comparison/synth_patients/coincidences_with_northcott2019.csv', index_col=0)
# Get unique genes as a list
northcott_genes = [northcott_genes[j].dropna().to_list() for j in northcott_genes.columns]
northcott_genes = [item for sublist in northcott_genes for item in sublist]
northcott_genes = list(set(northcott_genes))
northcott_genes

In [ ]:
# Plot differential genes data,clinical,genes,p_values_df,path_boxplot
plot_differential_genes(
    data=data.T, 
    clinical=metadata.replace({'Group3':'Group 3','Group4':'Group 4'}), 
    genes=northcott_genes, 
    p_values_df=p_values,
    path_boxplot=path_save_figs)

# Genes differentially expressed comparison with Nunez-Carpintero et al. 2021

In [ ]:
import os
import pandas as pd
from src.kruskalwallis_inbetween import plot_differential_genes

In [ ]:
path_save_figs = 'reports/figures/20241115_kw/boxplot_augmented/synth_patients/nunezcarpintero2021_genes'
os.makedirs(path_save_figs, exist_ok=True)

In [ ]:
# Load gene correspondence from original data:
gene_correspondence = pd.read_csv('data/raw/GEO/cavalli_gene_correspondence.csv', index_col=0)

In [ ]:
# Load augmented data
data = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_data.csv', index_col=0) 
metadata = pd.read_csv('data/interim/20241115_data_augmentation/real/augmented_clinical.csv', index_col=0).squeeze()
data.shape, metadata.shape

In [ ]:
# Select only synthetic patients
data = data.loc[metadata[metadata.str.contains('synthetic')].index]
metadata = metadata[metadata.str.contains('synthetic')].replace({'synthetic_Group 3':'Group3','synthetic_Group 4':'Group4','synthetic_G3-G4':'G3-G4'})
data.shape, metadata.shape

In [ ]:
metadata.value_counts()

In [ ]:
# Load p_values
p_values = pd.read_csv('data/processed/20241115_differentially_expressed_genes/synth_patients/p_values_dunn.csv', index_col=0)

In [ ]:
# Change column names from data, and indices from p_values, which are in ENSG format, to gene symbols:
data.columns=gene_correspondence.loc[data.columns]['HGNC_symbol_from_ensemblv77'].to_list()
p_values.index=gene_correspondence.loc[p_values.index]['HGNC_symbol_from_ensemblv77'].to_list()

In [ ]:
# Get genes differentially expressed that coincide with Nunez-Carpintero et al. 2021
nunez_genes = pd.read_csv('data/processed/20241115_genes_comparison/synth_patients/coincidences_with_external.csv', index_col=0)
# Get unique genes as a list
nunez_genes = set(nunez_genes['symbol'].to_list())
nunez_genes

In [ ]:
p_values.loc['ST8SIA2', 'g3_g4'], p_values.loc['ST8SIA2', 'g3_transition'], p_values.loc['ST8SIA2', 'g4_transition']

In [ ]:
# Plot differential genes data,clinical,genes,p_values_df,path_boxplot
plot_differential_genes(
    data=data.T, 
    clinical=metadata.replace({'Group3':'Group 3','Group4':'Group 4'}), 
    genes=nunez_genes, 
    p_values_df=p_values,
    path_boxplot=path_save_figs)

# Important genes from SHAP

In [ ]:
import pandas as pd

In [ ]:
shap_genes = pd.read_csv('data/interim/20241115_shap/real_data/selected_genes.csv', index_col=0).squeeze()
shap_genes.shape

In [ ]:
gene_correspondence = pd.read_csv('data/raw/GEO/cavalli_gene_correspondence.csv', index_col=0)


In [ ]:
shap_genes.head()

In [ ]:
shap_genes_correspondence = gene_correspondence.loc[shap_genes]
shap_genes_correspondence.shape

In [ ]:
shap_genes_correspondence.isna().sum()

In [ ]:
shap_genes_correspondence.sort_values('HGNC_symbol_from_ensemblv77')

In [ ]:
shap_genes_correspondence.sort_values('HGNC_symbol_from_ensemblv77')['HGNC_symbol_from_ensemblv77'].to_csv('data/processed/20241115_genes_comparison/synth_patients/important_genes_shap.csv')

In [ ]:
# Get genes differentially expressed that coincide with Nunez-Carpintero et al. 2021
nunez_genes = pd.read_csv('data/processed/20241115_genes_comparison/synth_patients/external_genes_equivalences.csv', index_col=0)
# Get unique genes as a list
# nunez_genes = set(nunez_genes['symbol'].to_list())
nunez_genes['symbol']

In [ ]:
coincidence_with_nunez=shap_genes_correspondence[shap_genes_correspondence['HGNC_symbol_from_ensemblv77'].isin(nunez_genes['symbol'])]
coincidence_with_nunez

In [ ]:
# Get genes differentially expressed that coincide with Northcott et al. 2019
northcott_genes = pd.read_csv('data/processed/20241115_genes_comparison/synth_patients/important_genes_northcott2019.csv', index_col=0)
# Get unique genes as a list
northcott_genes = [northcott_genes[j].dropna().to_list() for j in northcott_genes.columns]
northcott_genes = [item for sublist in northcott_genes for item in sublist]
northcott_genes = list(set(northcott_genes))
len(northcott_genes)

In [ ]:
coincidence_with_northcott=shap_genes_correspondence[shap_genes_correspondence['HGNC_symbol_from_ensemblv77'].isin(northcott_genes)]
coincidence_with_northcott

In [ ]:
shap_genes_correspondence['HGNC_symbol_from_ensemblv77'].isin(['MYC','SNCAIP']).sum()
shap_genes_correspondence['HGNC_symbol_from_ensemblv77'].isin(['TP53']).sum()

# Genes differentially expressed: finding the corresponding symbols

In [ ]:
import pandas as pd

In [ ]:
# Load gene correspondence from original data:
gene_correspondence = pd.read_csv('data/raw/GEO/cavalli_gene_correspondence.csv', index_col=0)
gene_correspondence.shape

In [ ]:
# Genes differentially expressed
diff_genes = pd.read_csv('data/processed/20241115_differentially_expressed_genes/synth_patients/always_diff_genes.csv', index_col=0).squeeze()
diff_genes.shape

In [ ]:
diff_genes_correspondence=gene_correspondence.loc[diff_genes]
diff_genes_correspondence.sort_values('HGNC_symbol_from_ensemblv77',inplace=True)
diff_genes_correspondence.shape

In [ ]:
diff_genes_correspondence.isna().sum()

In [ ]:
# What are the missing data?
diff_genes_correspondence[diff_genes_correspondence.isna().any(axis=1)]

In [ ]:
diff_genes_correspondence.head()

In [ ]:
diff_genes_correspondence['HGNC_symbol_from_ensemblv77'].to_csv('data/processed/20241115_differentially_expressed_genes/synth_patients/always_diff_genes_correspondence.csv')

# Check genes with high rec error after postprocessing (checked with Wasserstein)

In [ ]:
import pandas as pd

In [ ]:
# Load gene correspondence from original data:
gene_correspondence = pd.read_csv('data/raw/GEO/cavalli_gene_correspondence.csv', index_col=0)
gene_correspondence.shape

In [ ]:
genes_wasser = pd.read_csv('reports/figures/20241212_reconstruction_network_wasser/genes_above_1.csv',index_col=0).squeeze()
genes_wasser

In [ ]:
# Get corresponding symbols:
wasser_genes_correspondence=gene_correspondence.loc[genes_wasser]
wasser_genes_correspondence

In [ ]:
# Are any of this among shap genes? and among differentially expressed genes?
shap_genes = pd.read_csv('data/interim/20241115_shap/real_data/selected_genes.csv', index_col=0).squeeze()
diff_genes = pd.read_csv('data/processed/20241115_differentially_expressed_genes/synth_patients/always_diff_genes.csv', index_col=0).squeeze()
print(shap_genes.shape, diff_genes.shape)
genes_wasser.isin(shap_genes).sum(), genes_wasser.isin(diff_genes).sum()